In [78]:
import pandas as pd
import requests
import os
import cv2
import json
import numpy as np

vote_count_fpath = 'test/[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv'
vote_fpath = 'test/[WeVis] They Work for Us - Politician Data - [T] Votelog.csv'

In [2]:
images_p = [os.path.join('test/pdf/20190823113634A11', f) for f in os.listdir('test/pdf/20190823113634A11')]
images = [cv2.imread(f) for f in images_p]

In [3]:
def parse_text(image):
      gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
      blured = cv2.GaussianBlur(gray, (1, 9), 0)
      th, threshed = cv2.threshold(blured, 200, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
      threshed = cv2.dilate(threshed, kernel)
      hist = cv2.reduce(threshed, 1, cv2.REDUCE_AVG).reshape(-1)

      kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 2))
      dilated = cv2.dilate(threshed, kernel)
      contours, hier  = cv2.findContours(
            dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      rects = [cv2.boundingRect(c) for c in contours]
      rects.sort(key=lambda rect: rect[1])

      return rects


In [ ]:
os.path.abspath('')

In [24]:
pdf_dir = 'test/pdf'
images = {}
for dirpath, dir, filenames in os.walk(pdf_dir):
  if dirpath == pdf_dir: continue
  for fname in filenames:
    fpath = os.path.join(dirpath, fname)
    image = cv2.imread(fpath)
    images[fpath] = image



In [28]:
image.shape

(2105, 1498, 3)

In [46]:
dummy = {
          "original_width": 1498,
          "original_height": 2105,
          "image_rotation": 0,
          "value": {
            "x": 28,
            "y": 15,
            "width": 6,
            "height": 1,
            "rotation": 0,
            "rectanglelabels": [
              "text"
            ]
          },
          "from_name": "label",
          "to_name": "image",
          "type": "rectanglelabels",
          "origin": "manual"
        },

In [123]:
pred = []
size = []
ratio = []
id = 0
for fpath, image in images.items():
  results = []
  imid = 0
  boxes = parse_text(image)
  imh, imw, dept = image.shape
  for x, y, width, height in boxes:
    size.append(width*height)
    ratio.append(width/height)
    if width * height < 150 or width/height < 0.1: continue
    results.append({
          "original_width": imw,
          "original_height": imh,
          "image_rotation": 0,
          "value": {
            "x": (x/imw)*100,
            "y": (y)/imh*100,
            "width":width/imw*100,
            "height": height/imh*100,
            "rotation": 0,
            "rectanglelabels": [
              "text"
            ]
          },
          "from_name": "label",
          "to_name": "image",
          "type": "rectanglelabels",
        },)
    imid += 1
  imabspath = os.path.abspath(fpath)
  pred.append({
      'data': {'ocr': '/data/local-files/?d='+imabspath},
      'predictions': [{
        'model_version': '0.0.1',
        'id': id,
        'result': results,
      }]
    })
  id+=1
      

In [127]:
50/1000

0.05

In [126]:
np.array(ratio)

0.0079960019990005

In [108]:
with open('pred.json', 'w') as f:
  json.dump(pred, f, indent=3)

In [105]:
# im = images[0].copy()
# rects = parse_text(im)
# im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
# for x, y, w, h in rects:
#   x1 = x+w
#   y1 = y+h
#   im = cv2.rectangle(im, (x, y), (x1, y1), 1, 1)

In [106]:
# cv2.imwrite('rect.jpg', im)

# validate

In [110]:
size = []
for impred in pred:
  for prediction in impred['predictions']:
    new_res = []
    for result in prediction['result']:
      w = result['value']['width']
      h = result['value']['height']
      size.append(w*h)
      if w*h > 1:
        new_res.append(result)
    # annot['result'] = list()
    # annot['predictions'] = new_res

In [112]:
import numpy as np

In [116]:
size

[13.15134351740563,
 11.05194891684558,
 10.351093619679764,
 9.10477628128082,
 7.953597671003934,
 11.093175699031804,
 3.907030434879126,
 9.336280519711158,
 8.461004220988237,
 9.799288996571835,
 19.084828861284564,
 3.4281655033314418,
 3.3552258117711977,
 0.3551845849890114,
 13.395532919585573,
 16.19261152637404,
 12.215812690872072,
 7.611098249764532,
 3.3552258117711977,
 3.3552258117711977,
 9.012808844096165,
 3.51061906770389,
 44.09997177551066,
 3.2854574111483563,
 9.012808844096165,
 13.290880318651311,
 7.864801524756683,
 11.616438703703118,
 12.875441205851665,
 3.4281655033314418,
 3.5772161773893303,
 0.3805549124882266,
 47.66450278915038,
 30.92008663966841,
 7.8013757060086455,
 6.279156056055738,
 7.671352777575168,
 9.000123680346558,
 30.139949069067548,
 10.988523098097543,
 5.822490161069867,
 0.33298554842719824,
 0.3551845849890114,
 7.579385340390514,
 44.41075828737604,
 0.2663884387417586,
 31.69388162839447,
 40.61155174436858,
 23.88933463144842